In [178]:
import pandas as pd
import numpy as np
import names
from IPython.display import display, HTML
from sqlalchemy import create_engine

display(HTML("<style>.container { width:100% !important; }</style>"))

# Creating a db

In [120]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [121]:
%sql sqlite:///library.db

'Connected: @library.db'

# Creating tables

In [125]:
%%sql
CREATE TABLE User(
    userID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    age INTEGER NOT NULL,
    fines NUMERIC(5,2) DEFAULT 0,
    CHECK (age >= 7),
    CHECK (fines >= 0)
);

 * sqlite:///library.db
Done.


[]

In [126]:
%%sql
CREATE TABLE Librarian(
    librarianID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    salary NUMERIC(6,2) DEFAULT 0,
    department VARCHAR(20) NOT NULL,
    CHECK (salary >= 0)
);

 * sqlite:///library.db
Done.


[]

In [127]:
%%sql
CREATE TABLE LibraryItem(
    libraryItemID INTEGER PRIMARY KEY,
    author REFERENCES Item(author),
    itemName REFERENCES Item(itemName) NOT NULL,
    toBeAdded BOOLEAN
);

 * sqlite:///library.db
Done.


[]

In [128]:
%%sql
CREATE TABLE Item(
    author VARCHAR(30),
    itemName VARCHAR(30) NOT NULL,
    type VARCHAR(15) NOT NULL,
    PRIMARY KEY (author, itemName)
);

 * sqlite:///library.db
Done.


[]

In [129]:
%%sql
CREATE TABLE BorrowedItem(
    userID REFERENCES User(userID) NOT NULL,
    libraryItemID REFERENCES LibraryItem(libraryItemID) NOT NULL,
    dueDate DATETIME DEFAULT (date(CURRENT_TIMESTAMP, '+14 day')) NOT NULL,
    returnedDate DATETIME DEFAULT NULL,
    PRIMARY KEY (userID, libraryItemID, dueDate)
);

 * sqlite:///library.db
Done.


[]

In [130]:
%%sql
CREATE TABLE Event(
    startTS DATETIME NOT NULL,
    endTS DATETIME NOT NULL,
    room VARCHAR(10) NOT NULL,
    eventName VARCHAR(30) NOT NULL,
    audience VARCHAR(20),
    maxRegistrations INTEGER DEFAULT 0,
    artist VARCHAR(30),
    author REFERENCES Item(author),
    itemName REFERENCES Item(itemName),
    PRIMARY KEY (startTS, room),
    CHECK (maxRegistrations >= 0)
);

 * sqlite:///library.db
Done.


[]

In [131]:
%%sql
CREATE TABLE EventRegistration(
    startTS REFERENCES Event(startTS) NOT NULL,
    room REFERENCES Event(room) NOT NULL,
    userID REFERENCES User(userID) NOT NULL,
    PRIMARY KEY (startTS, room)
);

 * sqlite:///library.db
Done.


[]

# ASSERTIONS AND TRIGGERS

## List of Assertions

 - Maximum possible users <= some constant
 - Maximum employee salary <= budget

## List of Triggers

 - Users with fines cannot borrow items
 - Users cannot take items that are already checked out
 - Users cannot return items that are not part of the database
 - Users cannot register for events at max capacity
 - Users cannot register multiple times for the same event

# INDEXING

# Populating tables

## Table: Item

In [156]:
books = pd.read_csv('..//data//books2.csv')
books = books[['author', 'title']]
books.rename(columns={'title':'itemName'}, inplace=True)
books.loc[:,'type'] = 'book'
books.drop_duplicates(subset=['itemName'], inplace=True)

In [157]:
engine = create_engine('sqlite:///library.db', echo=False)
books.to_sql('Item', con=engine, if_exists='append', index=False)

49927

In [158]:
%%sql
SELECT * FROM Item LIMIT 5;

 * sqlite:///library.db
Done.


author,itemName,type
Suzanne Collins,The Hunger Games,book
"J.K. Rowling, Mary GrandPré (Illustrator)",Harry Potter and the Order of the Phoenix,book
Harper Lee,To Kill a Mockingbird,book
"Jane Austen, Anna Quindlen (Introduction)",Pride and Prejudice,book
Stephenie Meyer,Twilight,book


## Table: Librarian

In [196]:
numLibrarians = 50
departments = ['admin', 'catalog', 'circulation', 'infotech']
librarians = pd.DataFrame()
for librarianID in range(1,numLibrarians+1):
    firstName = names.get_first_name()
    lastName = names.get_last_name()
    age = np.random.randint(17,75)
    department = np.random.choice(departments, p=[0.2, 0.2, 0.4, 0.2])
    salary = 0
    if (age > 17):
        salary = np.random.randint(40000,90000)
        salary = round(salary, -3)
        
    librarian = pd.Series({'firstName':firstName, 'lastName':lastName, 'age':age, 'department':department, 'salary':salary})
    librarians = librarians.append(librarian, ignore_index=True)
    
numVolunteers = int(numLibrarians*0.10)
for num in range(0,numVolunteers):
    librarianID = np.random.randint(1,numLibrarians+1)
    librarians.loc[librarianID,'salary'] = 0
    
if ()

C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  librarians = librarians.append(librarian, ignore_index=True)
C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  librarians = librarians.append(librarian, ignore_index=True)
C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  librarians = librarians.append(librarian, ignore_index=True)
C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  libra

C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  librarians = librarians.append(librarian, ignore_index=True)
C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  librarians = librarians.append(librarian, ignore_index=True)
C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  librarians = librarians.append(librarian, ignore_index=True)
C:\Users\hatam\AppData\Local\Temp\ipykernel_24220\2420249230.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  libra

In [197]:
librarians

,firstName,lastName,age,department,salary
0,Francisco,Walkner,24,admin,53000
1,Steven,Prince,43,catalog,62000
2,Annette,Wright,24,circulation,89000
3,Juliana,Francisco,72,infotech,52000
4,Sheila,Simpson,61,admin,63000
5,Aaron,Steib,63,admin,42000
6,Mary,Easterling,63,admin,55000
7,Mari,Williams,24,infotech,40000
8,Carol,Hodge,18,catalog,61000
9,Samuel,Temple,36,infotech,78000


## Table: User

In [138]:
%%sql
DROP TABLE mytable

 * sqlite:///library.db
Done.


[]

In [118]:
books

,title,author,type
0,The Hunger Games,Suzanne Collins,book
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",book
2,To Kill a Mockingbird,Harper Lee,book
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",book
4,Twilight,Stephenie Meyer,book
...,...,...,...
52473,Fractured,Cheri Schmidt (Goodreads Author),book
52474,Anasazi,Emma Michaels,book
52475,Marked,Kim Richardson (Goodreads Author),book
52476,Wayward Son,"Tom Pollack (Goodreads Author), John Loftus (G...",book


In [185]:
%%sql
DROP TABLE Item;
DROP TABLE User;
DROP TABLE Librarian;
DROP TABLE LibraryItem;
DROP TABLE BorrowedItem;
DROP TABLE Event;
DROP TABLE EventRegistration;

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]